In [2]:
import numpy as np

from dask.dataframe import read_hdf

from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

from dask.distributed import Client

from dklearn import DaskGridSearchCV

%matplotlib inline

In [3]:
spy = read_hdf('/Users/armanarkilic/Desktop/spy.h5',
               'spy')
spy_design = read_hdf('/Users/armanarkilic/Desktop/spy_design.h5',
                      'spy_design')
spy.head()

,SPY.High,SPY.Low,SPY.Close,SPY.Volume
TIME,,,,
2008-01-02 09:31:00,146.645,146.535,146.645,591203.0
2008-01-02 09:32:00,146.685,146.635,146.685,385457.0
2008-01-02 09:33:00,146.775,146.680,146.775,343700.0
2008-01-02 09:34:00,146.755,146.705,146.725,863418.0
2008-01-02 09:35:00,146.775,146.685,146.775,457500.0


In [4]:
spy_design.head()

,variance,skew,hurst,rets_var
TIME,,,,
2008-01-02 09:31:00,0.000000,0.000000,0.000000,0.000000
2008-01-02 09:32:00,0.707107,-0.707107,0.000000,0.707107
2008-01-02 09:33:00,1.027297,-0.901304,0.000000,1.133169
2008-01-02 09:34:00,1.407897,1.277819,0.000000,-1.366479
2008-01-02 09:35:00,0.593341,-1.669053,-1.788854,0.854902


In [5]:
np.multiply(np.diff(spy.index.values), 10e-10).shape

(625424,)

In [6]:
re_turns = spy['SPY.Close']

logged_re_turns = np.log(re_turns)
logged_re_turns.head()
intermediate = np.diff(logged_re_turns)

intermediate = np.concatenate(([0], intermediate))
inx = np.array(spy.index.values, dtype=np.float64)


inx = np.array(np.multiply(np.diff(spy.index.values), 10e-10), dtype=np.float64)

inx = np.concatenate(([60], inx))

returns_running = intermediate / inx

def lag_xts(x_ts):
    tmp = x_ts[1:]
    return tmp


returns_advanced = lag_xts(returns_running)

In [7]:
returns_advanced.shape

(625424,)

In [12]:
advanced_ret = returns_advanced.reshape((returns_advanced.shape[0]))

In [19]:
spy_design.head()

,variance,skew,hurst,rets_var
TIME,,,,
2008-01-02 09:31:00,0.000000,0.000000,0.000000,0.000000
2008-01-02 09:32:00,0.707107,-0.707107,0.000000,0.707107
2008-01-02 09:33:00,1.027297,-0.901304,0.000000,1.133169
2008-01-02 09:34:00,1.407897,1.277819,0.000000,-1.366479
2008-01-02 09:35:00,0.593341,-1.669053,-1.788854,0.854902


In [58]:
# X = spy['2008': '2011'].shift(-1)
# y = spy_design['2008': '2011'].shift(-1)
# X = X.dropna()
# y = y.dropna()

X = spy.shift(-1).dropna()
y = advanced_ret
print(np.asarray(X).shape, y.shape)

print(X.head(), y[0:100])

(625424, 4) (625424,)
                     SPY.High  SPY.Low  SPY.Close  SPY.Volume
TIME                                                         
2008-01-02 09:31:00   146.685  146.635    146.685    385457.0
2008-01-02 09:32:00   146.775  146.680    146.775    343700.0
2008-01-02 09:33:00   146.755  146.705    146.725    863418.0
2008-01-02 09:34:00   146.775  146.685    146.775    457500.0
2008-01-02 09:35:00   146.795  146.725    146.735    416708.0 [  4.54550623e-06   1.02228586e-05  -5.67859176e-06   5.67859176e-06
  -4.54271862e-06  -1.59092729e-05  -3.41110657e-06   1.47764223e-05
   9.08667575e-06   2.27089508e-06   2.27058570e-06   1.30498682e-05
   1.70140366e-06   0.00000000e+00  -4.53746245e-06  -2.26919455e-06
   2.26919455e-06  -5.67356578e-06   0.00000000e+00  -7.94623817e-06
   0.00000000e+00   1.13540886e-06   5.67588436e-06   1.13494496e-06
  -5.67549780e-06  -7.94894521e-06  -1.36355896e-05   9.09163251e-06
   3.40808399e-06  -6.03123240e-05  -2.96671651e-05  -3.42653

In [51]:
print(np.asarray(X).shape,y.shape)

(625424, 4) (625424,)


In [52]:
client = Client('localhost:8900')
logistic = LogisticRegression()

pca = PCA(n_components=2)
cls = LogisticRegression() 


pipe = Pipeline([('pca', pca), ('logistic', cls)])
print(pca.get_params())
print(cls.get_params())

{'copy': True, 'iterated_power': 'auto', 'n_components': 2, 'random_state': None, 'svd_solver': 'auto', 'tol': 0.0, 'whiten': False}
{'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'max_iter': 100, 'multi_class': 'ovr', 'n_jobs': 1, 'penalty': 'l2', 'random_state': None, 'solver': 'liblinear', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


In [66]:
logistic = LogisticRegression()
pca = PCA()
pipe = Pipeline(steps=[('pca', pca),
                       ('logistic', logistic)])


#Parameters of pipelines can be set using ‘__’ separated parameter names:
grid = dict(pca__n_components=[1, 2, 3,4],
            logistic__C=[1e-2, 1.0, 10],
            logistic__penalty=['l1', 'l2'])

estimator = DaskGridSearchCV(pipe, grid)

In [67]:
estimator.fit(np.asarray(X), np.asarray(y))

distributed.utils - ERROR - Unknown label type: 'continuous'
Traceback (most recent call last):
  File "/Users/armanarkilic/anaconda/envs/dev/lib/python3.6/site-packages/distributed/utils.py", line 223, in f
    result[0] = yield make_coro()
  File "/Users/armanarkilic/anaconda/envs/dev/lib/python3.6/site-packages/tornado/gen.py", line 1015, in run
    value = future.result()
  File "/Users/armanarkilic/anaconda/envs/dev/lib/python3.6/site-packages/tornado/concurrent.py", line 237, in result
    raise_exc_info(self._exc_info)
  File "<string>", line 3, in raise_exc_info
  File "/Users/armanarkilic/anaconda/envs/dev/lib/python3.6/site-packages/tornado/gen.py", line 1021, in run
    yielded = self.gen.throw(*exc_info)
  File "/Users/armanarkilic/anaconda/envs/dev/lib/python3.6/site-packages/distributed/client.py", line 1156, in _gather
    traceback)
  File "/Users/armanarkilic/anaconda/envs/dev/lib/python3.6/site-packages/six.py", line 685, in reraise
    raise value.with_traceback(tb)


ValueError: Unknown label type: 'continuous'

In [ ]:
%debug pipe.fit(X, y)

NOTE: Enter 'c' at the ipdb>  prompt to continue execution.
> <string>(1)<module>()

ipdb> u
> /Users/armanarkilic/anaconda/envs/dev/lib/python3.6/bdb.py(431)run()
    429         sys.settrace(self.trace_dispatch)
    430         try:
--> 431             exec(cmd, globals, locals)
    432         except BdbQuit:
    433             pass

ipdb> u
*** Oldest frame


In [ ]:
debug

> <ipython-input-68-8b2a19b6cc6d>(1)<module>()
----> 1 debug
      2 

ipdb> y_type
*** NameError: name 'y_type' is not defined
ipdb> q


In [62]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

pca = PCA(n_components=2)
cls = LinearRegression(n_jobs=8) 

pipe = Pipeline([('pca', pca), ('linear', cls)])

In [63]:
pipe.fit(X, y)

Pipeline(steps=[('pca', PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('linear', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=8, normalize=False))])